## Composability (WIP)

`bodhilib` library is designed with composability in mind. It takes many ideas from strict functional languages like `Haskell` to design and implement its interface.

Using the bodhilib library, you can simplify the ingestion phase of your RAG process as follows:

In [ ]:
from fn import F # fn.py library

data_loader = get_data_loader("file")
spitter = get_splitter("sentence_splitter")
embedder = get_embedder("sentence_embedder")
vector_db = get_vector_db("qdrant", location=":memory:")

data_loader.add_resource(dir="./data")
data = data_loader.load()

result = F(data_loader.load) 
    >> F(splitter.split) 
    >> F(embedder.embed) 
    >> F(vector_db.upsert)

And to query your VectorDB, you can compose it like:

In [ ]:
query = "Who is the CEO of SpaceX?"
template = get_prompt_source("bodhiprompts").find("extractive_qna")

answer = (
    query
    >> F(embedder.embed)
    >> F(partial(vector_db.query, "articles_collection"))
    >> F(lambda nodes: [node.text for node in nodes])
    >> F(lambda nodes: {"context": "\n\n".join(nodes)})
    >> F(partial(template.to_prompt, query = query))
    >> F(llm.generate)
)
